## Limpieza de datos

La limpieza de datos es un paso crítico en el desarrollo de modelos de machine learning, ya que garantiza que los datos utilizados para entrenar y evaluar los modelos sean precisos, confiables y representativos, lo que a su vez mejora la calidad y la eficacia de los modelos resultantes.

In [3]:
import pandas as pd
from numpy import NaN
import requests

In [4]:
df = pd.read_csv(r"C:\Users\Andres\OneDrive\Escritorio\VisualizacionDatos\Proyecto\docs\raw_data.csv")
df = df.drop(columns='Unnamed: 0')
df.head()

C:\Users\Andres\AppData\Local\Temp\ipykernel_23036\146643001.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\Andres\OneDrive\Escritorio\VisualizacionDatos\Proyecto\docs\raw_data.csv")


,departamento,municipio,codigo_dane,armas_medios,fecha_hecho,genero,grupo_etario,cantidad
0,ATLÁNTICO,BARRANQUILLA (CT),8001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
1,BOYACÁ,DUITAMA,15238000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
2,CAQUETÁ,PUERTO RICO,18592000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
3,CASANARE,MANÍ,85139000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
4,CUNDINAMARCA,BOGOTÁ D.C. (CT),11001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1


Ajustes para columnas, valores nulos y duplicados

In [5]:
df = df.drop('codigo_dane', axis = 1).copy()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 575721 entries, 0 to 575720
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   departamento  575721 non-null  object
 1   municipio     575721 non-null  object
 2   armas_medios  575721 non-null  object
 3   fecha_hecho   575721 non-null  object
 4   genero        575721 non-null  object
 5   grupo_etario  574110 non-null  object
 6   cantidad      575721 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 30.7+ MB


In [7]:
empty_info = df.isnull().sum()*100/df.shape[0]
print(f'{empty_info}\n\n{df.isnull().sum()}')

departamento    0.000000
municipio       0.000000
armas_medios    0.000000
fecha_hecho     0.000000
genero          0.000000
grupo_etario    0.279823
cantidad        0.000000
dtype: float64

departamento       0
municipio          0
armas_medios       0
fecha_hecho        0
genero             0
grupo_etario    1611
cantidad           0
dtype: int64


In [8]:
df = df[~df['grupo_etario'].isnull()]

Eliminando duplicados

In [9]:
df = df.loc[:, ~df.columns.duplicated()].copy()

Cambio tipo de datos

In [10]:
df['genero'] = df['genero'].astype('category')
df['grupo_etario'] = df['grupo_etario'].astype('category')
df['armas_medios'] = df['armas_medios'].astype('category')
df['fecha_hecho'] = pd.to_datetime(df['fecha_hecho'], format = '%d/%m/%Y')

In [11]:
df.dtypes

departamento            object
municipio               object
armas_medios          category
fecha_hecho     datetime64[ns]
genero                category
grupo_etario          category
cantidad                 int64
dtype: object

Se modifica la codificación de la columna departamento para que no haya errores

In [12]:
df.loc[:, 'departamento'] = df['departamento'].str.normalize('NFKD').str.encode('ascii', errors = 'ignore').str.decode('utf-8')

In [13]:
df['departamento'].replace({'SAN ANDRES':'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
                             'VALLE':'VALLE DEL CAUCA',
                             'NARINO':'NARIÑO',
                             'GUAJIRA':'LA GUAJIRA'}, inplace = True)
df.departamento.unique()

C:\Users\Andres\AppData\Local\Temp\ipykernel_23036\3136357760.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['departamento'].replace({'SAN ANDRES':'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',


array(['ATLANTICO', 'BOYACA', 'CAQUETA', 'CASANARE', 'CUNDINAMARCA',
       'SUCRE', 'VALLE DEL CAUCA', 'HUILA', 'ANTIOQUIA', 'ARAUCA',
       'BOLIVAR', 'CALDAS', 'CAUCA', 'CESAR', 'CHOCO', 'CORDOBA',
       'MAGDALENA', 'META', 'NARIÑO', 'NORTE DE SANTANDER', 'PUTUMAYO',
       'RISARALDA', 'SANTANDER', 'TOLIMA', 'VAUPES', 'GUAVIARE',
       'LA GUAJIRA', 'QUINDIO', 'AMAZONAS', 'VICHADA', 'GUAINIA',
       'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
       'NO REPORTA'], dtype=object)

Se eliminan datos sin informe de departamento

In [14]:
df['departamento'].replace('NO REPORTA', NaN, inplace = True)
df = df[~df['departamento'].isnull()]

Se cambia el nombre del municipio de Bogotá y su departamento para trazar mapas más adelante

In [15]:
df.loc[df['municipio'] == 'BOGOTÁ D.C. (CT)', 'department'] = 'SANTAFE DE BOGOTA D.C'

In [16]:
# Change on genre no reports
df['genero'] = df['genero'].replace('NO REPORTADO', 'NO REPORTA')

# Change on age_group no reports
df['grupo_etario'] = df['grupo_etario'].replace('NO REPORTADO', 'NO REPORTA')

# Change on gun_type based on Penal Code
df['armas_medios'] = df['armas_medios'].replace({'ARMA BLANCA / CORTOPUNZANTE':'ARMA BLANCA',
                            '-':'NO REPORTA',
                            'NO REPORTADO':'NO REPORTA',
                            'CORTOPUNZANTES':'ARMA BLANCA',
                            'CORTANTES':'ARMA BLANCA',
                            'CONTUNDENTES':'ARMA BLANCA',
                            'PUNZANTES':'ARMA BLANCA'})

C:\Users\Andres\AppData\Local\Temp\ipykernel_23036\1952285717.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df['genero'] = df['genero'].replace('NO REPORTADO', 'NO REPORTA')
C:\Users\Andres\AppData\Local\Temp\ipykernel_23036\1952285717.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df['grupo_etario'] = df['grupo_etario'].replace('NO REPORTADO', 'NO REPORTA')
C:\Users\Andres\AppData\Local\Temp\ipykernel_23036\1952285717.py:8: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, re

In [17]:
print(df['armas_medios'].unique(), 
      df['genero'].unique(), 
      df['grupo_etario'].unique())

['ARMA BLANCA', 'ARMA DE FUEGO', 'NO REPORTA', 'SIN EMPLEO DE ARMAS', 'ESCOPOLAMINA']
Categories (5, object): ['ARMA BLANCA', 'ARMA DE FUEGO', 'ESCOPOLAMINA', 'NO REPORTA', 'SIN EMPLEO DE ARMAS'] ['MASCULINO', 'FEMENINO', 'NO REPORTA']
Categories (3, object): ['FEMENINO', 'MASCULINO', 'NO REPORTA'] ['ADULTOS', 'ADOLESCENTES', 'MENORES', 'NO REPORTA']
Categories (4, object): ['ADOLESCENTES', 'ADULTOS', 'MENORES', 'NO REPORTA']


Valores atípicos

Como se vio anteriormente y dado que cada fila en el conjunto de datos representa un solo registro, considero apropiado eliminar registros donde los casos sean más de 20, ya que es absurdo pensar que un solo incidente de violencia doméstica ha sido reportado con más de 20 víctimas de violencia en un mismo lugar. Esto podría haber sido causado por un error en la entrada de datos, ya que hay casos que superan los 100.

In [18]:
df[['cantidad']].query('cantidad > 20').count()

cantidad    3439
dtype: int64

In [19]:
df = df.query('cantidad < 20').reset_index(drop=True).copy()

In [20]:
df.to_parquet('data_cleaned.parquet', index=None)
df.to_csv('data_cleaned.csv', index=None)